In [5]:
import os
import math

depth_dir = "/data/holly_jackson/Make3D/Test134Img/"
bash_file = "run_all_make3d_test.sh"

num_procs = os.cpu_count()
print("Number of processors:", num_procs)

max_parallel = max(1, num_procs // 2)

# list all training data and extract image number
files = sorted([
    f[:-4] for f in os.listdir(depth_dir) if f.endswith(".jpg") # f[:-4]
])
print(files)

num_batches = math.ceil(len(files) / max_parallel)

batch = 0
with open(bash_file, "w") as f:
    f.write("#!/bin/bash\n\n")
    f.write(f"echo 'Starting batch {batch+1}/{num_batches}'\n")

    for i, num in enumerate(files, 1):
        f.write(f"python run_coordinate_descent.py {num} &\n")
        # insert a wait every max_parallel jobs
        if i % max_parallel == 0:
            f.write("wait\n\n")
            batch += 1
            f.write(f"echo 'Starting batch {batch+1}/{num_batches}'\n")

    # final wait for leftover jobs
    f.write("wait\n\n")

print(f"Wrote {bash_file} with {len(files)} jobs in {num_batches} parallel batches.")

Number of processors: 144
['img-060705-17.10.14-p-049t000', 'img-060705-17.10.14-p-080t000', 'img-060705-17.10.14-p-316t000', 'img-060705-17.28.10-p-161t000', 'img-060705-17.35.48-p-254t000', 'img-060705-17.35.48-p-285t000', 'img-060705-17.43.27-p-111t000', 'img-060705-17.51.18-p-018t000', 'img-060705-17.51.18-p-111t000', 'img-10.21op2-p-015t000', 'img-10.21op2-p-139t000', 'img-10.21op2-p-313t000', 'img-10.21op3-p-283t000', 'img-10.21op6-p-139t000', 'img-10.21op6-p-220t000', 'img-10.21op6-p-313t000', 'img-10.21op9-p-251t000', 'img-10.21op9-p-343t000', 'img-combined1-p-108t0', 'img-combined1-p-170t0', 'img-combined1-p-220t0', 'img-combined1-p-251t0', 'img-gatesback1-p-139t0', 'img-gatesback2-p-313t0', 'img-gatesback4-p-108t0', 'img-gatesback4-p-139t0', 'img-manmade5-p-169t0', 'img-manmade5-p-313t0', 'img-manmade6-p-77t0', 'img-math1-p-282t0', 'img-math11-p-343t0', 'img-math12-p-108t0', 'img-math12-p-189t0', 'img-math12-p-343t0', 'img-math13-p-139t0', 'img-math13-p-344t0', 'img-math13-p-

654
